<a href="https://colab.research.google.com/github/penelopetir/BIOL74-Final-Project/blob/main/Final_Project_Test_Pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, we'll install neuron in our current runtime as usual.

In [ ]:
%pip install neuron # only need to run this cell once to install
                    # neuron in the local jupyter environment

Run the code block below just once to get all the files from the repository into our colab session and compile the MOD mechanism files we'll be using

In [ ]:
repo_name = 'BIOL74-Final-Project'
if 'google.colab' in str(get_ipython()):
    import os
    if not os.path.exists(repo_name):
        !git clone https://github.com/penelopetir/{repo_name}.git # downloads repository into our Google colab session's file system

    os.chdir(repo_name) # Changing working directory to downloaded repository

# Compile mechanisms
!nrnivmodl mechanisms

/content/BIOL74-Final-Project/BIOL74-Final-Project
Mod files: "mechanisms/mechanisms/CadepK.mod" "mechanisms/mechanisms/Ca.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


## Introduction
For this exercise, we'll be adapting a model of a L5PC to model granule neurons traced from wild type and PTEN KO groups


Start by importing the packages we'll be using.

In [ ]:
from neuron import h # all NEURON hoc functions are accessed through h
from neuron import gui # if you're running this notebook in a local environment (instead of on google colab), launches the GUI
h.load_file('stdrun.hoc') # loads neuron standard hoc library, not always necessary
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if 'google.colab' in str(get_ipython()): # For making interactive plots work on google colab
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    plotly.offline.init_notebook_mode() # for printing notebooks offline

#Importing Morphologies

Files from the github of traced neurons from NeuroLuicda 360 are imported and created with proper morphology. Biophysics are modeled after the Aradi et al 1999 paper.

In [ ]:
from helper_functions import createGranuleCell

genotype = "KO" #change this string to "WT" or "KO" to determine which neuron to run the following code on

if genotype == "WT":
  WT_file = "morphologies/WT.asc" # morphology file
  GC = createGranuleCell(WT_file, "passive")
elif genotype == "KO":
  KO_file = "morphologies/PTEN_KO.asc" # morphology file
  GC = createGranuleCell(KO_file, "passive")
else:
  print("No neuron found")

# Create recording vectors for time and somatic voltage
t_vec = h.Vector().record(h._ref_t)
v_soma = h.Vector().record(GC.soma[0](0.5)._ref_v)

somastack_process 7
center
194.973	-263.9	-6	
Principle values
0.619399	37.2071	344.619	
Principle axes
 0.914    0.406    -0.0147 
 -0.402   0.9      -0.166  
 -0.054   0.157    0.986   
stk_triang_area = 781.997


Importing tau from previous file

In [ ]:

vclamp = h.SEClamp(GC.soma[0](0.5))
vclamp.dur1 = 40
vclamp.amp1 = -70

vclamp.dur2 = 40
vclamp.amp2 = -60

vclamp.dur3 = 40
vclamp.amp3 = -70

t = h.Vector().record(h._ref_t) # time vector
I = h.Vector().record(vclamp._ref_i) # voltage clamp current (equal to membrane current)

In [ ]:
h.v_init = vclamp.amp1 # mV - initial voltage of the model system
h.dt = 0.0125  # ms - time step
h.celsius = 37 # degrees celsius - temperature of simulation
# choose a tstop
h.tstop = 130 # ms - simulation duration

In [ ]:
h.run()

i = np.array(I)
fig = make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t,y=i),row=1,col=1)
fig['layout']['yaxis1']['title']= 'Current Density (nA)'
fig['layout']['xaxis1']['title']= 'time (ms)'
fig.update_layout(yaxis=dict(range=[-5,5]))

fig.show()
